# PBS with sgkit

This notebook is for running a PBS scan using sgkit, to reproduce the scikit-allel one (`pbs_scans.ipynb`).

You need to have run `sgkit_import.ipynb` first to convert the data into sgkit format.

In [1]:
%run setup.ipynb

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from dask.diagnostics import ProgressBar
import sgkit as sg
import xarray as xr

First, let's inspect the input data. Note that it has a single chunk in the `samples` dimension, which is a requirement for running the popgen analyses.

In [4]:
ds = sg.load_dataset(here() / 'data/sgkit/ag1000g.zarr')
ds

<xarray.Dataset>
Dimensions:             (alleles: 4, ploidy: 2, samples: 1142, variants: 57837885)
Dimensions without coordinates: alleles, ploidy, samples, variants
Data variables:
    call_genotype       (variants, samples, ploidy) int8 dask.array<chunksize=(524288, 1142, 2), meta=np.ndarray>
    call_genotype_mask  (variants, samples, ploidy) bool dask.array<chunksize=(524288, 1142, 2), meta=np.ndarray>
    sample_id           (samples) <U1 dask.array<chunksize=(1142,), meta=np.ndarray>
    variant_allele      (variants, alleles) |S1 dask.array<chunksize=(4194304, 4), meta=np.ndarray>
    variant_contig      (variants) int32 dask.array<chunksize=(4194304,), meta=np.ndarray>
    variant_position    (variants) int32 dask.array<chunksize=(4194304,), meta=np.ndarray>
Attributes:
    contigs:                    ['2L', '2R', '3L', '3R', 'X']
    max_variant_allele_length:  1

## Cohorts

We need to divide the samples into separate cohorts, which we get from the `pop_defs` YAML:

In [5]:
cohort_ids = list(pop_defs.keys())
cohort_ids

['ao_col',
 'bf_col',
 'bf_gam',
 'ci_col',
 'cm_sav_gam',
 'fr_gam',
 'ga_gam',
 'gh_col',
 'gh_gam',
 'gm',
 'gn_gam',
 'gq_gam',
 'gw',
 'ke',
 'ug_gam']

In [6]:
ds["cohort_id"] = xr.DataArray(cohort_ids, dims="cohorts")
ds

<xarray.Dataset>
Dimensions:             (alleles: 4, cohorts: 15, ploidy: 2, samples: 1142, variants: 57837885)
Dimensions without coordinates: alleles, cohorts, ploidy, samples, variants
Data variables:
    call_genotype       (variants, samples, ploidy) int8 dask.array<chunksize=(524288, 1142, 2), meta=np.ndarray>
    call_genotype_mask  (variants, samples, ploidy) bool dask.array<chunksize=(524288, 1142, 2), meta=np.ndarray>
    sample_id           (samples) <U1 dask.array<chunksize=(1142,), meta=np.ndarray>
    variant_allele      (variants, alleles) |S1 dask.array<chunksize=(4194304, 4), meta=np.ndarray>
    variant_contig      (variants) int32 dask.array<chunksize=(4194304,), meta=np.ndarray>
    variant_position    (variants) int32 dask.array<chunksize=(4194304,), meta=np.ndarray>
    cohort_id           (cohorts) <U10 'ao_col' 'bf_col' ... 'ke' 'ug_gam'
Attributes:
    contigs:                    ['2L', '2R', '3L', '3R', 'X']
    max_variant_allele_length:  1

Sample metadata is in the `df_samples` dataframe, so we can use that to produce a mapping from sample to cohort

In [7]:
sample_cohorts = np.full_like(ds.sample_id.values, -1, dtype=np.int8)
for i, pop in enumerate(cohort_ids):
    pop_query = (
            pop_defs[pop]['query']
            .replace('region', 'location')
            .replace('Gado-Badzere', 'Gado Badzere')
            .replace('Zembe-Borongo', 'Zembe Borongo')
    )
    loc_pop = df_samples.query(pop_query).index.values
    sample_cohorts[loc_pop] = i
sample_cohorts

array([7, 7, 7, ..., 3, 3, 3], dtype=int8)

Add `sample_cohort` to the dataset

In [8]:
ds["sample_cohort"] = xr.DataArray(sample_cohorts, dims="samples")
ds

<xarray.Dataset>
Dimensions:             (alleles: 4, cohorts: 15, ploidy: 2, samples: 1142, variants: 57837885)
Dimensions without coordinates: alleles, cohorts, ploidy, samples, variants
Data variables:
    call_genotype       (variants, samples, ploidy) int8 dask.array<chunksize=(524288, 1142, 2), meta=np.ndarray>
    call_genotype_mask  (variants, samples, ploidy) bool dask.array<chunksize=(524288, 1142, 2), meta=np.ndarray>
    sample_id           (samples) <U1 dask.array<chunksize=(1142,), meta=np.ndarray>
    variant_allele      (variants, alleles) |S1 dask.array<chunksize=(4194304, 4), meta=np.ndarray>
    variant_contig      (variants) int32 dask.array<chunksize=(4194304,), meta=np.ndarray>
    variant_position    (variants) int32 dask.array<chunksize=(4194304,), meta=np.ndarray>
    cohort_id           (cohorts) <U10 'ao_col' 'bf_col' ... 'ke' 'ug_gam'
    sample_cohort       (samples) int8 7 7 7 7 7 8 7 8 7 7 ... 3 3 3 3 3 3 3 3 3
Attributes:
    contigs:                    ['2L', '2R', '3L', '3R', 'X']
    max_variant_allele_length:  1

Some samples are not in any of the named cohorts, and have -1 in the `sample_cohort` variable. These are ignored in cohort allele counts.

## Count cohort alleles

Rather than just computing PBS directly, we are going to do the computation for allele counts separately, since it is a fairly expensive computation which we can save to disk so we don't have to do it repeatedly.

In [11]:
def checkpoint_dataset(ds, path, data_vars=None):
    if data_vars is not None:
        ds = ds.drop_vars(set(ds.data_vars) - set(data_vars))
    sg.save_dataset(ds, path, mode="a")
    return sg.load_dataset(path)

In [12]:
if not sg.variables.cohort_allele_count in ds:
    ds = sg.count_cohort_alleles(ds)
    with ProgressBar():
        ds = sg.checkpoint_dataset(ds, here() / 'data/sgkit/ag1000g.zarr', [sg.variables.cohort_allele_count])

[########################################] | 100% Completed | 14min 18.3s


The technique used here computes a new variable and checkpoints it to disk. This ensures that downstream operations won't need to recompute the variable.

In [13]:
ds

<xarray.Dataset>
Dimensions:              (alleles: 4, cohorts: 15, ploidy: 2, samples: 1142, variants: 57837885)
Dimensions without coordinates: alleles, cohorts, ploidy, samples, variants
Data variables:
    call_genotype        (variants, samples, ploidy) int8 dask.array<chunksize=(524288, 1142, 2), meta=np.ndarray>
    call_genotype_mask   (variants, samples, ploidy) bool dask.array<chunksize=(524288, 1142, 2), meta=np.ndarray>
    cohort_allele_count  (variants, cohorts, alleles) int32 dask.array<chunksize=(524288, 15, 4), meta=np.ndarray>
    sample_id            (samples) <U1 dask.array<chunksize=(1142,), meta=np.ndarray>
    variant_allele       (variants, alleles) |S1 dask.array<chunksize=(4194304, 4), meta=np.ndarray>
    variant_contig       (variants) int32 dask.array<chunksize=(4194304,), meta=np.ndarray>
    variant_position     (variants) int32 dask.array<chunksize=(4194304,), meta=np.ndarray>
Attributes:
    contigs:                    ['2L', '2R', '3L', '3R', 'X']
    max_variant_allele_length:  1

## Subset to segregating variants

Next we filter out variants that are not informative. The slightly tricky part about this step is that it is cohort-dependent, so we see if each variant is segregating _within the cohort triple_.

In [15]:
seg_zarr_path = here() / 'data/sgkit/ag1000g_segregating.zarr'

In [17]:
ds = ds.assign_coords({"cohorts_0": cohort_ids, "cohorts_1": cohort_ids, "cohorts_2": cohort_ids})
cohort_triple = ("ao_col", "ga_gam", "gw")
index = ds.indexes["cohorts_0"]
cohorts = np.array([index.get_loc(id) for id in cohort_triple])
cohorts

array([ 0,  6, 12])

In [18]:
cohort_triple_mask = np.full((len(index),), False)
cohort_triple_mask[cohorts] = True
cohort_triple_mask = xr.DataArray(cohort_triple_mask, dims=("cohorts"))
cohort_triple_mask

<xarray.DataArray (cohorts: 15)>
array([ True, False, False, False, False, False,  True, False, False,
       False, False, False,  True, False, False])
Dimensions without coordinates: cohorts

In [19]:
def is_segregating(ds):
    ac = ds.cohort_allele_count.where(cohort_triple_mask).sum(dim="cohorts")
    return np.sum(ac > 0, axis=-1) > 1

In [21]:
ds_seg = ds.assign(segregating=lambda ds: is_segregating(ds))
ds_seg = ds_seg.drop(["call_genotype", "call_genotype_mask"]) # don't need these vars any more
ds_seg

<xarray.Dataset>
Dimensions:              (alleles: 4, cohorts: 15, cohorts_0: 15, cohorts_1: 15, cohorts_2: 15, samples: 1142, variants: 57837885)
Coordinates:
  * cohorts_0            (cohorts_0) <U10 'ao_col' 'bf_col' ... 'ke' 'ug_gam'
  * cohorts_1            (cohorts_1) <U10 'ao_col' 'bf_col' ... 'ke' 'ug_gam'
  * cohorts_2            (cohorts_2) <U10 'ao_col' 'bf_col' ... 'ke' 'ug_gam'
Dimensions without coordinates: alleles, cohorts, samples, variants
Data variables:
    cohort_allele_count  (variants, cohorts, alleles) int32 dask.array<chunksize=(524288, 15, 4), meta=np.ndarray>
    sample_id            (samples) <U1 dask.array<chunksize=(1142,), meta=np.ndarray>
    variant_allele       (variants, alleles) |S1 dask.array<chunksize=(4194304, 4), meta=np.ndarray>
    variant_contig       (variants) int32 dask.array<chunksize=(4194304,), meta=np.ndarray>
    variant_position     (variants) int32 dask.array<chunksize=(4194304,), meta=np.ndarray>
    segregating          (variants) bool dask.array<chunksize=(524288,), meta=np.ndarray>
Attributes:
    contigs:                    ['2L', '2R', '3L', '3R', 'X']
    max_variant_allele_length:  1

In [22]:
with ProgressBar():
    # TODO: why does this make so many passes over the data? https://github.com/pystatgen/sgkit/issues/299
    ds_seg = ds_seg.sel(variants=ds_seg.segregating)
    ds_seg = ds_seg.chunk({"variants": 524288}) # rechunk to uniform chunk sizes so we can save to zarr
    sg.save_dataset(ds_seg, seg_zarr_path, mode="w")

[########################################] | 100% Completed | 38.4s
[########################################] | 100% Completed | 39.3s
[########################################] | 100% Completed | 40.2s
[########################################] | 100% Completed | 40.2s
[########################################] | 100% Completed | 40.1s
[########################################] | 100% Completed | 40.4s
[########################################] | 100% Completed | 48.2s


## Windowing

To compute popgen stats we need to set up windows along the genome. For PBS we are just going to have half-overlapping windows of size 500 variants.

In [23]:
ds_seg = sg.load_dataset(seg_zarr_path)
ds = sg.window(ds_seg, size=500, step=250)
ds

<xarray.Dataset>
Dimensions:              (alleles: 4, cohorts: 15, cohorts_0: 15, cohorts_1: 15, cohorts_2: 15, samples: 1142, variants: 25181146, windows: 100727)
Coordinates:
  * cohorts_0            (cohorts_0) <U10 'ao_col' 'bf_col' ... 'ke' 'ug_gam'
  * cohorts_1            (cohorts_1) <U10 'ao_col' 'bf_col' ... 'ke' 'ug_gam'
  * cohorts_2            (cohorts_2) <U10 'ao_col' 'bf_col' ... 'ke' 'ug_gam'
Dimensions without coordinates: alleles, cohorts, samples, variants, windows
Data variables:
    window_contig        (windows) int64 0 0 0 0 0 0 0 0 0 ... 4 4 4 4 4 4 4 4 4
    window_start         (windows) int64 0 250 500 ... 25180724 25180974
    window_stop          (windows) int64 500 750 1000 ... 25181146 25181146
    cohort_allele_count  (variants, cohorts, alleles) int32 dask.array<chunksize=(524288, 15, 4), meta=np.ndarray>
    sample_id            (samples) <U1 dask.array<chunksize=(1142,), meta=np.ndarray>
    segregating          (variants) bool dask.array<chunksize=(524288,), meta=np.ndarray>
    variant_allele       (variants, alleles) |S1 dask.array<chunksize=(524288, 4), meta=np.ndarray>
    variant_contig       (variants) int32 dask.array<chunksize=(524288,), meta=np.ndarray>
    variant_position     (variants) int32 dask.array<chunksize=(524288,), meta=np.ndarray>
Attributes:
    contigs:                    ['2L', '2R', '3L', '3R', 'X']
    max_variant_allele_length:  1

## PBS

We are now in a position to calculate the PBS statistic. The following computes the statistic for a single cohort triples. (Note we could have computed PBS for all cohorts by omitting the `cohorts` keyword, however the segregating sites and windows are different for each triple, which is why we compute each one independently.)

In [24]:
ds = ds.assign_coords({"cohorts_0": cohort_ids, "cohorts_1": cohort_ids, "cohorts_2": cohort_ids})
pbs = sg.pbs(ds, cohorts=[("ao_col", "ga_gam", "gw")], merge=False)
pbs

,Array,Chunk
Bytes,2.72 GB,56.65 MB
Shape,"(100727, 15, 15, 15)","(2098, 15, 15, 15)"
Count,794 Tasks,49 Chunks
Type,float64,numpy.ndarray


In [25]:
with ProgressBar():
    pbs = pbs.chunk({"windows": 65536}) # rechunk to uniform chunk sizes so we can save to zarr
    sg.save_dataset(pbs, here() / 'data/sgkit/ag1000g_pbs.zarr', mode="w")

[########################################] | 100% Completed |  2min  4.5s


In [26]:
pbs = sg.load_dataset(here() / 'data/sgkit/ag1000g_pbs.zarr')
pbs = pbs.assign_coords({"cohorts_0": list(pop_defs), "cohorts_1": list(pop_defs), "cohorts_2": list(pop_defs)})
pbs

<xarray.Dataset>
Dimensions:    (cohorts_0: 15, cohorts_1: 15, cohorts_2: 15, windows: 100727)
Coordinates:
  * cohorts_0  (cohorts_0) <U10 'ao_col' 'bf_col' 'bf_gam' ... 'ke' 'ug_gam'
  * cohorts_1  (cohorts_1) <U10 'ao_col' 'bf_col' 'bf_gam' ... 'ke' 'ug_gam'
  * cohorts_2  (cohorts_2) <U10 'ao_col' 'bf_col' 'bf_gam' ... 'ke' 'ug_gam'
Dimensions without coordinates: windows
Data variables:
    stat_pbs   (windows, cohorts_0, cohorts_1, cohorts_2) float64 dask.array<chunksize=(65536, 15, 15, 15), meta=np.ndarray>

Have a look at the PBS values for the given cohort triple:

In [27]:
pbs["stat_pbs"].sel(cohorts_0="ao_col", cohorts_1="ga_gam", cohorts_2="gw")[:100].values

array([ 6.68421170e-04, -2.98647987e-02, -6.88631588e-03,  4.11808673e-02,
        2.93915826e-02,  4.47354647e-02,  5.32595306e-02,  5.33295520e-03,
        1.25369405e-04, -2.34907708e-04,  2.34255131e-02,  3.59365242e-02,
        6.78268377e-03,  6.30811771e-03,  1.66361368e-02,  5.51133392e-04,
        3.78090586e-02,  3.77681136e-02,  2.18148091e-02,  1.69955168e-02,
        1.66234560e-02,  3.73869401e-02,  2.93010285e-02,  6.24751324e-02,
        6.64477695e-02,  7.22526922e-02,  6.58134420e-02,  1.11473390e-02,
        7.38049452e-02,  7.43922126e-02,  8.25989207e-02,  1.39932378e-01,
        1.61962710e-01,  1.12635638e-01,  6.84851400e-02,  1.35928800e-01,
        1.56276254e-01,  1.32666031e-01,  1.68125382e-01,  1.94662428e-01,
        1.73045162e-01,  1.21253156e-01,  1.02039742e-01,  1.30498702e-01,
        1.24894507e-01,  1.13136120e-01,  1.61295239e-01,  1.75011401e-01,
        1.79389843e-01,  1.59514218e-01,  6.28970007e-02,  3.46677788e-02,
        6.77630747e-02,  

## Concordance with sckit-allel

We'll compare chromosome X since it doesn't have any of the arm stiching or other complications of chromosomes 2 and 3.

In [28]:
ds2 = xr.merge([ds, pbs]).assign_attrs(ds.attrs)
contig_index = ds2.attrs["contigs"].index("X")
X_windows = ds2.window_contig == contig_index
dsX = ds2.sel(windows=X_windows)
dsX

<xarray.Dataset>
Dimensions:              (alleles: 4, cohorts: 15, cohorts_0: 15, cohorts_1: 15, cohorts_2: 15, samples: 1142, variants: 25181146, windows: 9164)
Coordinates:
  * cohorts_0            (cohorts_0) <U10 'ao_col' 'bf_col' ... 'ke' 'ug_gam'
  * cohorts_1            (cohorts_1) <U10 'ao_col' 'bf_col' ... 'ke' 'ug_gam'
  * cohorts_2            (cohorts_2) <U10 'ao_col' 'bf_col' ... 'ke' 'ug_gam'
Dimensions without coordinates: alleles, cohorts, samples, variants, windows
Data variables:
    window_contig        (windows) int64 4 4 4 4 4 4 4 4 4 ... 4 4 4 4 4 4 4 4 4
    window_start         (windows) int64 22890224 22890474 ... 25180724 25180974
    window_stop          (windows) int64 22890724 22890974 ... 25181146 25181146
    cohort_allele_count  (variants, cohorts, alleles) int32 dask.array<chunksize=(524288, 15, 4), meta=np.ndarray>
    sample_id            (samples) <U1 dask.array<chunksize=(1142,), meta=np.ndarray>
    segregating          (variants) bool dask.array<chunksize=(524288,), meta=np.ndarray>
    variant_allele       (variants, alleles) |S1 dask.array<chunksize=(524288, 4), meta=np.ndarray>
    variant_contig       (variants) int32 dask.array<chunksize=(524288,), meta=np.ndarray>
    variant_position     (variants) int32 dask.array<chunksize=(524288,), meta=np.ndarray>
    stat_pbs             (windows, cohorts_0, cohorts_1, cohorts_2) float64 dask.array<chunksize=(9164, 15, 15, 15), meta=np.ndarray>
Attributes:
    contigs:                    ['2L', '2R', '3L', '3R', 'X']
    max_variant_allele_length:  1

In [29]:
stat_pbs = dsX["stat_pbs"].sel(cohorts_0="ao_col", cohorts_1="ga_gam", cohorts_2="gw").values
stat_pbs

array([0.33624208, 0.3322171 , 0.31385064, ..., 0.7021624 , 0.66886772,
       0.6288948 ])

Load PBS data calculated using the `pbs_scans.ipynb` notebook:

In [30]:
pbs_root = zarr.open(str(here() / 'data/gwss/pbs/pbs.zarr'))

In [31]:
def get_scikit_allel_pbs(pop1, pop2, pop3, chromosome, window_size=100, window_step=100, normed=True, markersize=1, min_maf=None, 
            physical=True, genetic=True, fig_scale=1e-7, plot_scaled=True):

    # setup zarr group to store data
    grp_path = f'/{pop1}_{pop2}_{pop3}/{window_size}/{window_step}/{chromosome}'
    grp = pbs_root.require_group(grp_path)
    complete = grp.attrs.get('complete', False)

    if complete:
        # previously run, load from zarr
        windows = grp['windows'][:]
        gwindows = grp['gwindows'][:]
        pbs = grp['pbs'][:]
        pbs_scaled = grp['pbs_scaled'][:]
        
        return windows, gwindows, pbs, pbs_scaled
        
    return None

In [32]:
ska_windows, ska_gwindows, ska_pbs, ska_pbs_scaled = get_scikit_allel_pbs("ao_col", "ga_gam", "gw", "X", 500, 250)

Are they equal?

In [33]:
import numpy as np
np.testing.assert_allclose(stat_pbs[:-2], ska_pbs) # skip last two windows (bug in scikit-allel?)